# This script will use Evolutionary Algorithm to produce most slow fixating graphs 

In [ ]:
# imports
%load_ext autoreload
%autoreload 2

import numpy as np
import joblib
import pandas as pd
from population_graph import PopulationGraph
from analysis.analysis_utils import GRAPH_PROPERTY_COLUMNS
from pathlib import Path
from tqdm import tqdm



In [ ]:
SEED = 42
rng = np.random.default_rng(SEED)

INITIAL_GRAPH_POPULATION = 10
NUMBER_OF_CHILDREN = 10
GENERATIONS = 100
N_NODES = 31
N_EDGES = 34

In [ ]:
def add_new_random_graph(graph_zoo: list[PopulationGraph], 
                         wl_set:set, 
                         n_nodes:int, 
                         n_edges:int, 
                         name:str, 
                         seed=None):
    
    new_graph, new_wl = None, None
    while(new_wl is None or new_wl in wl_set):
        new_graph = PopulationGraph.random_connected_graph(n_nodes, n_edges, name=name, seed=seed)
        new_wl = new_graph.wl_hash
    graph_zoo.append(new_graph)
    wl_set.add(new_wl)
    return wl_set


In [ ]:
lr = joblib.load('./ml_models/linear_regression_pipeline.joblib')

In [ ]:
# Create some random graphs

graph_zoo:list[PopulationGraph] = []
wl_set = set()

for i in range(INITIAL_GRAPH_POPULATION):
    # add_new_random_graph(graph_zoo, wl_set, N_NODES, N_EDGES, name=f"random-{i}", seed=int(rng.integers(0, 2**32)))
    new_graph, new_wl = None, None
    while(new_wl is None or new_wl in wl_set):
        new_graph = PopulationGraph.random_connected_graph(N_NODES, N_EDGES, name=f"random-{i}", seed=None)
        new_wl = new_graph.wl_hash
    graph_zoo.append(new_graph)
    wl_set.add(new_wl)


In [ ]:


def run_evolutionary_search(
    initial_population: list[PopulationGraph],
    model, 
    generations: int = 50, 
    pop_size: int = 10,
    n_children: int = 50,  # Generate more children than parents to explore more
    objective: str = "maximize", # 'maximize' for slow fixation, 'minimize' for fast
    rng: np.random.Generator = None
):
    """
    Runs a (mu + lambda) evolutionary strategy.
    """
    if rng is None:
        rng = np.random.default_rng(42)

    # 1. Initialize State
    current_pop = initial_population.copy()
    wl_set = set([g.wl_hash for g in current_pop])
    
    # We maintain a cache of properties to avoid re-calculating for survivors
    # Key: wl_hash, Value: dict of properties
    prop_cache = {} 
    
    # History for plotting
    history = {'best_fitness': [], 'avg_fitness': []}

    print(f"Starting Evolution: {generations} generations, optimization: {objective}")

    for gen in tqdm(range(generations), desc="Evolving"):
        
        # --- A. REPRODUCTION ---
        # Generate children from current population
        children = []
        # We try to generate n_children, ensuring they are unique
        
        max_attempts = 10
        
        for parent in current_pop:
            for i in range(n_children):
                attempts = 0
                while attempts < max_attempts: 
                    seed = rng.integers(0, 2**32)
                    parent_name = parent.name
                    new_name = f'{parent_name.split("_")[0]}_gen_{gen}' 
                    # Use your efficiently defined mutate function here
                    child = parent.mutate_graph(seed=seed, name=new_name)
                    
                    # Verify uniqueness
                    if child.wl_hash not in wl_set:
                        children.append(child)
                        wl_set.add(child.wl_hash)
                        break
                    attempts += 1

        # --- B. EVALUATION ---
        # Pool everyone together
        candidates = current_pop + children
        
        # Calculate properties ONLY for those not in cache
        new_props_list = []
        new_graphs_to_process = []
        
        for g in candidates:
            if g.wl_hash not in prop_cache:
                new_graphs_to_process.append(g)

        # Batch calculation for new graphs
        if new_graphs_to_process:
            for g in new_graphs_to_process:
                # Assuming calculate_graph_properties returns a dict
                props = g.calculate_graph_properties()
                prop_cache[g.wl_hash] = props

        # Construct DataFrame for Prediction
        # We assume GRAPH_PROPERTY_COLUMNS is available globally or passed in
        all_props = [prop_cache[g.wl_hash] for g in candidates]
        
        X = pd.DataFrame(all_props)
        # Ensure columns match training data exactly
        X = X[GRAPH_PROPERTY_COLUMNS].select_dtypes(include=[np.number])
        if "density" in X.columns:
            X = X.drop(columns=["density"])
            
        # Predict Fitness (Fixation Time)
        fitness_scores = model.predict(X)
        
        # Attach scores to graph objects for sorting (or zip them)
        scored_candidates = list(zip(candidates, fitness_scores))

        # --- C. SELECTION ---
        # Sort based on objective
        if objective == "maximize":
            # Descending order (Best = Slowest fixation)
            scored_candidates.sort(key=lambda x: x[1], reverse=True)
        else:
            # Ascending order (Best = Fastest fixation)
            scored_candidates.sort(key=lambda x: x[1], reverse=False)

        # Keep the top 'pop_size'
        survivors = scored_candidates[:pop_size]
        current_pop = [g for g, score in survivors]
        
        # Logging
        best_score = survivors[0][1]
        avg_score = np.mean([s[1] for s in survivors])
        history['best_fitness'].append(best_score)
        history['avg_fitness'].append(avg_score)
        
        # Optional: Prune wl_set to save memory? 
        # Actually, keep it to ensure we never revisit bad ancestors.

    return current_pop, history

In [ ]:
run_name = "accelerator_graphs"
CATEGORY = "Accelerator"

# 1. Configuration
params = {
    "initial_population": graph_zoo, # Start with your random zoo
    "model": lr,                     # Your trained Linear Regression
    "generations": GENERATIONS,               # How long to run
    "pop_size": INITIAL_GRAPH_POPULATION,                  # Keep top 10 elite graphs
    "n_children": NUMBER_OF_CHILDREN,                # Generate 30 new mutatesd graphs per graph in the population
    "objective": "minimize",         # We want SLOW fixation
    "rng": rng
}

# 2. Run
final_pop, history = run_evolutionary_search(**params)

# 3. Analyze Results
import matplotlib.pyplot as plt

# Plot Learning Curve
plt.figure(figsize=(10, 5))
plt.plot(history['best_fitness'], label="Best Fitness (Slowest)")
plt.plot(history['avg_fitness'], label="Avg Fitness", linestyle="--")
plt.xlabel("Generation")
plt.ylabel("Predicted Fixation Time")
plt.title("Evolution of Respiratory Topologies")
plt.legend()
plt.show()

# 4. View the Champion
champion = final_pop[0]
print(f"Champion Graph Name: {champion.name}")
print(f"Predicted Fixation Time: {history['best_fitness'][-1]:.2f}")

# Visualize the champion (assuming you have a plot method)
# nx.draw(champion.graph, with_labels=True)


output_dir = Path("tmp_winning_graphs")
output_dir.mkdir(parents=True, exist_ok=True)

for graph in final_pop:
    graph.category = CATEGORY

joblib.dump(final_pop, output_dir / f"{run_name}_zoo.joblib")

In [ ]:
for graph in graph_zoo:
    graph.draw()

In [ ]:
for graph in final_pop:
    graph.draw()